### Dataset explorations -- full customer service records


In [31]:
import duckdb
import pandas as pd
import os.path

home = os.path.expanduser("~")

csr_file_path = os.path.join(home, "misc", "seattle-data", "csr-all.csv")
conn = duckdb.connect()

# Get the first 10 rows of the CSV file
csr_result = conn.execute(f"SELECT * FROM read_csv('{csr_file_path}') LIMIT 10")

# Print the result -- do not wrap columns
pd.set_option('display.expand_frame_repr', False)
print(csr_result.fetchdf())

  Service Request Number                        Service Request Type                            City Department        Created Date         Method Received              Status                                   Location       X_Value        Y_Value   Latitude   Longitude                 Latitude/Longitude  ZIP Code  Council District Police Precinct     Neighborhood
0            21-00000006  Street Sign and Traffic Signal Maintenance  SDOT-Seattle Department of Transportation 2021-01-01 05:24:02     Find It Fix It Apps            Reported      2052 RAINIER AVE S, SEATTLE, WA 98144  1.277820e+06  216740.084565  47.584693 -122.302683  POINT (-122.30268292 47.58469328)     98144                 3           SOUTH         ATLANTIC
1            21-00000007                                    Graffiti               SPU-Seattle Public Utilities 2021-01-01 07:31:00     Find It Fix It Apps            Reported       2920 DEXTER AVE N, SEATTLE, WA 98109  1.266760e+06  239220.678480  47.645724 -122.34

In [32]:
# Create a set() of all possible `Status` values
result = conn.execute(f"SELECT DISTINCT Status FROM read_csv('{csr_file_path}')")
statuses = set(result.fetchdf()["Status"].tolist())
for status in sorted(list(statuses)):
	print(status)


Canceled
Closed
Closed -Incomplete Information
Closed as Duplicate
Duplicate (Closed)
Duplicate (Open)
New
Open
Reported
Transferred to Other Dept


In [33]:
# Create a set() of all `Service Request Type` values
result = conn.execute(f"SELECT DISTINCT \"Service Request Type\" FROM read_csv('{csr_file_path}')")
service_request_types = set(result.fetchdf()["Service Request Type"].tolist())
for service_request_type in sorted(list(service_request_types)):
	print(service_request_type)

ADA Request (Transportation)
Abandoned Vehicle
Abandoned Vehicle/72hr Parking Ordinance
Animal Noise
Bicycle Facility Maintenance
Business Related Complaint
Business Violation of Public Health Requirements
Clogged Storm Drain
Damaged Sidewalk
Dead Animal
Feedback about the Customer Service Requests Portal
Feedback about the Find It, Fix It mobile app
Found a Pet
General Inquiry - Animal Shelter
General Inquiry - City Light
General Inquiry - Customer Service Bureau
General Inquiry - Police Department
General Inquiry - Public Utilities
General Inquiry - Transportation
Graffiti
Illegal Dumping / Needles
Internet/Cable Issue
Lost a Pet
Nightlife Noise Complaint
Nuisance dogs in a park
Overgrown Vegetation
Parking Enforcement
Parks and Recreation Maintenance
Pollution Report Form
Pothole
Public Garage or Parking Lot Complaint
Public Litter and Recycling Cans
Public Utilities Direct Mail Opt Out
Safe Routes to School
Scooter or Bike Share Issue
Seattle Center
Snow and Ice
Street Sign Mainten

In [34]:
police_file_path = os.path.join(home, "misc", "seattle-data", "911.csv")
conn = duckdb.connect()

# Get the first 10 rows of the CSV file
police_result = conn.execute(f"SELECT * FROM read_csv('{police_file_path}') LIMIT 10")

# Print the result -- do not wrap columns
pd.set_option('display.expand_frame_repr', False)
print(police_result.fetchdf())

   CAD Event Number                 Event Clearance Description                 Call Type  Priority                           Initial Call Type                                  Final Call Type Original Time Queued        Arrived Time Precinct  Sector Beat  Blurred_Longitude  Blurred_Latitude
0     2021000303215    UNABLE TO LOCATE INCIDENT OR COMPLAINANT  TELEPHONE OTHER, NOT 911         5                            TRU - HARASSMENT                 --ASSAULTS - HARASSMENT, THREATS  2021-11-15 11:40:24 2021-11-15 11:42:15     EAST  EDWARD   E1        -122.323977         47.617749
1     2011000271127                  REPORT WRITTEN (NO ARREST)  TELEPHONE OTHER, NOT 911         3                           NARCOTICS - FOUND         --SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON  2011-08-18 13:13:08 2011-08-18 18:41:57    NORTH    NORA   N1        -122.354156         47.709618
2     2023000340425                         ASSISTANCE RENDERED                    ONVIEW         3     SUSPICIOUS PERSON

In [35]:
# Create a set() of all `Initial Call Type` values
result = conn.execute(f"SELECT DISTINCT \"Initial Call Type\" FROM read_csv('{police_file_path}')")
initial_call_types = set(result.fetchdf()["Initial Call Type"].tolist())
for initial_call_type in sorted(list(initial_call_types)):
	print(initial_call_type)

--ALARM-COMM (INC BANK, ATM, SCHOOLS, BSN)
--ALARM-RESIDENTIAL PANIC OR DURESS
--ASSAULTS - HARASSMENT, THREATS
--ASSAULTS, OTHER
--AUTOMOBILES - AUTO THEFT
--BURGLARY - NON RESIDENTIAL/COMMERCIAL
--BURGLARY - RESIDENTIAL OCCUPIED
--CRISIS COMPLAINT - GENERAL
--CROWD MGMNT (STAND BY ONLY)
--DISTURBANCE - OTHER
--DV - ARGUMENTS, DISTURBANCE (NO ARREST)
--DV - DOMESTIC THREATS BY PHONE OR WRITING
--DV - DOMESTIC VIOL/ASLT (ARREST MANDATORY)
--FRAUD - FRAUD, INCLUDING BUNCO
--HARBOR - VESSEL, RECOVERY (THEFT)
--MISCHIEF OR NUISANCE - GENERAL
--NARCOTICS - NARS REPORT
--NARCOTICS - OTHER
--PERSON - MISSING PERSON
--PERSON - RUNAWAY
--PROPERTY - FOUND PROPERTY
--PROPERTY DEST (DAMG)
--ROBBERY - STRONG ARM
--SEX OFFENSES (NON-RAPE) - LEWD CONDUCT
--THEFT - ALL OTHER
--THEFT - CAR PROWL
--THEFT - SHOPLIFT
--TRAFFIC - BLOCKING TRAFFIC
--TRAFFIC - MV COLLISION INVESTIGATION
--TRAFFIC - PARKING VIOL (EXCEPT ABANDONED CAR)
--WARRANT SERVICES - FELONY
--WEAPON, PERSON WITH - GUN
-ASSIGNED DUTY - C

In [36]:
# Create a set() of all `Priority` values
result = conn.execute(f"SELECT DISTINCT Priority FROM read_csv('{police_file_path}')")
priorities = set(result.fetchdf()["Priority"].tolist())
for priority in sorted(list(priorities)):
	print(priority)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
nan


In [37]:
# How many pri-1 calls were there in 2024 and 2025?
# Use the `Original Time Queued` column, which is a TIMESTAMP, to filter the data (year >= 2024)
result = conn.execute(f"SELECT COUNT(*) FROM read_csv('{police_file_path}') WHERE Priority <= 2 AND EXTRACT(year FROM \"Original Time Queued\") >= 2024")
print(result.fetchdf())

   count_star()
0        143634


In [38]:
# Figure out what the average difference bwteen `Arrived Time` and `Original Time Queued` is for each priority level
result = conn.execute(f"""
	SELECT 
		Priority, 
		AVG(EXTRACT(epoch FROM "Arrived Time" - "Original Time Queued") / 60) AS avg_difference_minutes 
	FROM read_csv('{police_file_path}') 
	GROUP BY Priority
""")
tuples = sorted([(row.Priority, row.avg_difference_minutes) for row in result.fetchdf().itertuples()], key=lambda x: x[0])
for priority, avg_difference_minutes in tuples:
	print(priority, round(avg_difference_minutes, 1))

1.0 8.9
2.0 36.6
3.0 55.5
4.0 65.0
5.0 82.1
6.0 65.0
7.0 0.0
8.0 66.7
9.0 0.2
nan 0.0


In [41]:
output_csv_path = os.path.join(home, "misc", "seattle-data", "911-2024plus-pri1or2.csv")

# Load the data 
result = conn.execute(f"""
	SELECT * 
	FROM read_csv('{police_file_path}') 
	WHERE Priority <= 2 AND EXTRACT(year FROM "Original Time Queued") >= 2024
""")

# Convert the dataframe as follows: keep only the columns:
# - Arrived Time
# - Priority
# - Original Time Queued
# - Initial Call Type
# - Blurred_Latitude
# - Blurred_Longitude

selected_df = result.fetchdf()[["Priority", "Original Time Queued", "Initial Call Type", "Blurred_Latitude", "Blurred_Longitude"]]

# Now rename:
# - Original Time Queued -> Created Date
# - Blurred_Latitude -> Latitude
# - Blurred_Longitude -> Longitude
# And create a new column called `Location` that only contains the string "Seattle, WA" for all rows:

modified_df = selected_df.rename(columns={"Original Time Queued": "Created Date", "Blurred_Latitude": "Latitude", "Blurred_Longitude": "Longitude"})
modified_df["Location"] = "Seattle, WA"

modified_df.to_csv(output_csv_path, index=False)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))